In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
from functools import reduce

pd.options.display.float_format = "{:,.2f}".format

In [3]:
avgtemp_file = "Resources\GlobalLandTemperaturesByCountry.csv"
popdf_file = "Resources\population_total_long.csv"

avgtemp_df = pd.read_csv(avgtemp_file)
popdf = pd.read_csv(popdf_file)

In [4]:
avgtemp_df.dropna()

avgtemp_df=avgtemp_df.dropna()

In [5]:
avgtemp_df['dt'] = pd.to_datetime(avgtemp_df['dt'])
avgtemp_df.head()


,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.38,2.29,Åland
5,1744-04-01,1.53,4.68,Åland
6,1744-05-01,6.70,1.79,Åland
7,1744-06-01,11.61,1.58,Åland
8,1744-07-01,15.34,1.41,Åland


In [6]:
avgtemp_df['Day'] = avgtemp_df['dt'].dt.day
avgtemp_df['Month'] = avgtemp_df['dt'].dt.month
avgtemp_df['Year'] = avgtemp_df['dt'].dt.year

avgtemp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Day,Month,Year
0,1743-11-01,4.38,2.29,Åland,1,11,1743
5,1744-04-01,1.53,4.68,Åland,1,4,1744
6,1744-05-01,6.70,1.79,Åland,1,5,1744
7,1744-06-01,11.61,1.58,Åland,1,6,1744
8,1744-07-01,15.34,1.41,Åland,1,7,1744


In [7]:
avgtemp_df = avgtemp_df[(avgtemp_df.Year >= 1963) & (avgtemp_df.Year <= 2013)]

avgtemp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Day,Month,Year
2630,1963-01-01,-6.29,0.98,Åland,1,1,1963
2631,1963-02-01,-8.08,0.59,Åland,1,2,1963
2632,1963-03-01,-5.51,0.56,Åland,1,3,1963
2633,1963-04-01,1.45,0.74,Åland,1,4,1963
2634,1963-05-01,9.56,0.43,Åland,1,5,1963


In [8]:
atdf = avgtemp_df.groupby( [ "Year", "Country"] ) ['AverageTemperature'].mean().to_frame().reset_index()

atdf.head()

,Year,Country,AverageTemperature
0,1963,Afghanistan,15.03
1,1963,Africa,24.27
2,1963,Albania,13.09
3,1963,Algeria,23.32
4,1963,American Samoa,26.86


In [9]:
atdf.head()

,Year,Country,AverageTemperature
0,1963,Afghanistan,15.03
1,1963,Africa,24.27
2,1963,Albania,13.09
3,1963,Algeria,23.32
4,1963,American Samoa,26.86


In [10]:
merged_df = pd.merge(popdf, atdf,  how='inner', left_on=['Country Name','Year'], right_on = ['Country','Year'])

In [11]:
merged_df=merged_df[["Country","Year","Count","AverageTemperature"]]


In [12]:
merged_df.head()

,Country,Year,Count,AverageTemperature
0,Aruba,1963,56695,28.32
1,Afghanistan,1963,9543205,15.03
2,Angola,1963,5679458,21.84
3,Albania,1963,1762621,13.09
4,Andorra,1963,16412,10.64


In [13]:
# Filter to exclude undesired years
# Frequency: 10 years
# rename Count column to Population
new_df = merged_df[merged_df['Year'] % 10 == 3]
new_df = new_df.rename(columns={'Count':'Population'})
new_df.head()

,Country,Year,Population,AverageTemperature
0,Aruba,1963,56695,28.32
1,Afghanistan,1963,9543205,15.03
2,Angola,1963,5679458,21.84
3,Albania,1963,1762621,13.09
4,Andorra,1963,16412,10.64


In [14]:
# Arrange the data to caclculate the cascading difference
new_df = new_df.sort_values(["Country", "Year"], ascending = (True, True))

In [15]:
new_df.reset_index(drop=True, inplace=True)


In [16]:
# new_df["new_column"] = 1
# new_df

In [17]:
new_df["Average_temp_diff"] = new_df["AverageTemperature"].diff().fillna(0)
new_df["Population_diff"] = new_df["Population"].diff().fillna(0)

In [18]:
new_df

,Country,Year,Population,AverageTemperature,Average_temp_diff,Population_diff
0,Afghanistan,1963,9543205,15.03,0.00,0.00
1,Afghanistan,1973,12108963,14.24,-0.80,"2,565,758.00"
2,Afghanistan,1983,12537730,14.61,0.38,"428,767.00"
3,Afghanistan,1993,15816603,14.44,-0.18,"3,278,873.00"
4,Afghanistan,2003,23680871,14.92,0.48,"7,864,268.00"
...,...,...,...,...,...,...
992,Zimbabwe,1973,5877726,21.46,0.70,"1,699,000.00"
993,Zimbabwe,1983,8254747,22.37,0.91,"2,377,021.00"
994,Zimbabwe,1993,11092766,21.85,-0.51,"2,838,019.00"
995,Zimbabwe,2003,11982224,21.89,0.04,"889,458.00"


In [19]:
# create pivot table for population
Pop_df = pd.pivot_table(new_df, index=["Country"], columns=["Year"], values=["Population_diff"])
Pop_df

Population_diff                                            \
Year                      1963          1973          1983          1993   
Country                                                                    
Afghanistan               0.00  2,565,758.00    428,767.00  3,278,873.00   
Albania         -30,506,968.00    534,131.00    547,208.00    383,327.00   
Algeria           9,017,711.00  3,797,022.00  5,392,050.00  6,533,640.00   
American Samoa  -38,118,098.00      7,066.00      7,312.00     14,608.00   
Andorra             -39,301.00     11,820.00     12,635.00     20,104.00   
...                        ...           ...           ...           ...   
Uruguay        -313,422,601.00    186,991.00    150,919.00    204,709.00   
Uzbekistan        6,039,467.00  3,711,891.00  4,036,867.00  4,764,336.00   
Vietnam           5,478,017.00 10,882,308.00 11,544,859.00 14,151,924.00   
Zambia          -87,393,368.00  1,259,442.00  1,862,370.00  2,174,570.00   
Zimbabwe        -10,747,778.00  1,699,000.00  2,377,021.00  2,838,019.00   

                                           
Year                    2003         2013  
Country                                    
Afghanistan     7,864,268.00 8,588,718.00  
Albania          -187,671.00  -144,524.00  
Algeria         4,628,642.00 5,875,975.00  
American Samoa      8,484.00    -3,791.00  
Andorra            12,211.00     7,592.00  
...                      ...          ...  
Uruguay           145,923.00    65,771.00  
Uzbekistan      3,625,650.00 4,675,550.00  
Vietnam        10,001,348.00 8,451,816.00  
Zambia          2,600,257.00 3,669,761.00  
Zimbabwe          889,458.00 1,368,132.00  

[167 rows x 6 columns]

In [20]:
# rename index and create country as a column
Poptable = Pop_df.reset_index()
Poptable

Country Population_diff                              \
Year                            1963          1973          1983   
0        Afghanistan            0.00  2,565,758.00    428,767.00   
1            Albania  -30,506,968.00    534,131.00    547,208.00   
2            Algeria    9,017,711.00  3,797,022.00  5,392,050.00   
3     American Samoa  -38,118,098.00      7,066.00      7,312.00   
4            Andorra      -39,301.00     11,820.00     12,635.00   
..               ...             ...           ...           ...   
162          Uruguay -313,422,601.00    186,991.00    150,919.00   
163       Uzbekistan    6,039,467.00  3,711,891.00  4,036,867.00   
164          Vietnam    5,478,017.00 10,882,308.00 11,544,859.00   
165           Zambia  -87,393,368.00  1,259,442.00  1,862,370.00   
166         Zimbabwe  -10,747,778.00  1,699,000.00  2,377,021.00   

                                               
Year          1993          2003         2013  
0     3,278,873.00  7,864,268.00 8,588,718.00  
1       383,327.00   -187,671.00  -144,524.00  
2     6,533,640.00  4,628,642.00 5,875,975.00  
3        14,608.00      8,484.00    -3,791.00  
4        20,104.00     12,211.00     7,592.00  
..             ...           ...          ...  
162     204,709.00    145,923.00    65,771.00  
163   4,764,336.00  3,625,650.00 4,675,550.00  
164  14,151,924.00 10,001,348.00 8,451,816.00  
165   2,174,570.00  2,600,257.00 3,669,761.00  
166   2,838,019.00    889,458.00 1,368,132.00  

[167 rows x 7 columns]

In [21]:
# rename columns to sanitise table
Poptable.columns = ['Country', '1963', '1973', '1983', '1993','2003','2013']
Poptable


,Country,1963,1973,1983,1993,2003,2013
0,Afghanistan,0.00,"2,565,758.00","428,767.00","3,278,873.00","7,864,268.00","8,588,718.00"
1,Albania,"-30,506,968.00","534,131.00","547,208.00","383,327.00","-187,671.00","-144,524.00"
2,Algeria,"9,017,711.00","3,797,022.00","5,392,050.00","6,533,640.00","4,628,642.00","5,875,975.00"
3,American Samoa,"-38,118,098.00","7,066.00","7,312.00","14,608.00","8,484.00","-3,791.00"
4,Andorra,"-39,301.00","11,820.00","12,635.00","20,104.00","12,211.00","7,592.00"
...,...,...,...,...,...,...,...
162,Uruguay,"-313,422,601.00","186,991.00","150,919.00","204,709.00","145,923.00","65,771.00"
163,Uzbekistan,"6,039,467.00","3,711,891.00","4,036,867.00","4,764,336.00","3,625,650.00","4,675,550.00"
164,Vietnam,"5,478,017.00","10,882,308.00","11,544,859.00","14,151,924.00","10,001,348.00","8,451,816.00"
165,Zambia,"-87,393,368.00","1,259,442.00","1,862,370.00","2,174,570.00","2,600,257.00","3,669,761.00"


In [22]:
# create pivot table for Average Temp

ATemp_df= pd.pivot_table(new_df, index=["Country"], columns=["Year"], values=["Average_temp_diff"])
ATemp_df


Average_temp_diff                              
Year                        1963  1973  1983  1993  2003  2013
Country                                                       
Afghanistan                 0.00 -0.80  0.38 -0.18  0.48  1.62
Albania                    -3.45 -0.60  0.02  0.61  0.50  1.37
Algeria                     8.32 -0.43  0.80 -0.23  0.94  0.73
American Samoa              1.74  0.28 -0.21 -0.11  0.45  0.25
Andorra                   -16.88  0.54  0.82 -0.56  1.47 -0.59
...                          ...   ...   ...   ...   ...   ...
Uruguay                     6.21 -0.23  0.13  0.24 -0.04 -0.85
Uzbekistan                 -3.11 -0.73  1.18 -2.13  1.20  3.03
Vietnam                     7.57  0.58 -0.40 -0.01  0.56  0.73
Zambia                     -4.22  0.46  0.87 -0.63  0.45 -0.96
Zimbabwe                   -0.44  0.70  0.91 -0.51  0.04 -1.18

[167 rows x 6 columns]

In [23]:
# reset index so country is a column
ATemp = ATemp_df.reset_index()
ATemp

Country Average_temp_diff                              
Year                              1963  1973  1983  1993  2003  2013
0        Afghanistan              0.00 -0.80  0.38 -0.18  0.48  1.62
1            Albania             -3.45 -0.60  0.02  0.61  0.50  1.37
2            Algeria              8.32 -0.43  0.80 -0.23  0.94  0.73
3     American Samoa              1.74  0.28 -0.21 -0.11  0.45  0.25
4            Andorra            -16.88  0.54  0.82 -0.56  1.47 -0.59
..               ...               ...   ...   ...   ...   ...   ...
162          Uruguay              6.21 -0.23  0.13  0.24 -0.04 -0.85
163       Uzbekistan             -3.11 -0.73  1.18 -2.13  1.20  3.03
164          Vietnam              7.57  0.58 -0.40 -0.01  0.56  0.73
165           Zambia             -4.22  0.46  0.87 -0.63  0.45 -0.96
166         Zimbabwe             -0.44  0.70  0.91 -0.51  0.04 -1.18

[167 rows x 7 columns]

In [24]:
# rename columns to sanitise table
ATemp.columns = ['Country', '1963', '1973', '1983', '1993','2003','2013']
ATemp


,Country,1963,1973,1983,1993,2003,2013
0,Afghanistan,0.00,-0.80,0.38,-0.18,0.48,1.62
1,Albania,-3.45,-0.60,0.02,0.61,0.50,1.37
2,Algeria,8.32,-0.43,0.80,-0.23,0.94,0.73
3,American Samoa,1.74,0.28,-0.21,-0.11,0.45,0.25
4,Andorra,-16.88,0.54,0.82,-0.56,1.47,-0.59
...,...,...,...,...,...,...,...
162,Uruguay,6.21,-0.23,0.13,0.24,-0.04,-0.85
163,Uzbekistan,-3.11,-0.73,1.18,-2.13,1.20,3.03
164,Vietnam,7.57,0.58,-0.40,-0.01,0.56,0.73
165,Zambia,-4.22,0.46,0.87,-0.63,0.45,-0.96


# SQL Schema

In [25]:
engine = create_engine('sqlite:///save_pandas.db', echo=True)
sqlite_connection = engine.connect()

In [26]:
sqlite_table = "Pop"
Poptable.to_sql(sqlite_table, sqlite_connection, if_exists='replace')


2021-10-12 21:58:20,024 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Pop")
2021-10-12 21:58:20,029 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-12 21:58:20,032 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Pop")
2021-10-12 21:58:20,034 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-12 21:58:20,036 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-10-12 21:58:20,038 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-12 21:58:20,040 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Pop")
2021-10-12 21:58:20,041 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-12 21:58:20,045 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-10-12 21:58:20,047 INFO sqlalchemy.engine.Engine [raw sql] ('Pop',)
2021-10-12 21:58:20,050 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Pop")
2021-10-12 21:58:2

In [27]:
sqlite_table = "Temp"
ATemp.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-10-12 21:58:20,670 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Temp")
2021-10-12 21:58:20,671 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-12 21:58:20,673 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Temp")
2021-10-12 21:58:20,674 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-12 21:58:20,676 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-10-12 21:58:20,676 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-12 21:58:20,679 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Temp")
2021-10-12 21:58:20,680 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-12 21:58:20,682 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-10-12 21:58:20,683 INFO sqlalchemy.engine.Engine [raw sql] ('Temp',)
2021-10-12 21:58:20,685 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Temp")
2021-10-12 21

In [28]:
sqlite_connection.close()